In [1]:
import sys,os
sys.path.append("../")

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from IPython.display import HTML
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy import stats
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
r_thrshld = 5
k_thrshld = 5

## reading the change object and clustering.

In [3]:
article_name = "Truth"
change_object_dir =  "../data/change objects/"
filename =  f"{article_name}_change.h5"

change_object_file = os.path.join(change_object_dir, filename)

In [4]:
if os.path.exists(change_object_file):
    with pd.HDFStore(change_object_file, 'r') as store:
        change_object_dataframe = store.get("data")
else:
    print("file do not exist")
change_object_dataframe.shape

(31777, 12)

## Reduce the change object.

In [6]:
inner_join_change_object = change_object_dataframe[(change_object_dataframe["ins_tokens"]!=()) & (change_object_dataframe["del_tokens"]!=())]
display(inner_join_change_object.shape)
inner_join_change_object.head(2)

(11562, 14)

ins_start_pos  \
from revision id to revision id timestamp           timegap           editor                    
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122    2           33.0   
                                                                             3           36.0   

                                                                                ins_end_pos  \
from revision id to revision id timestamp           timegap           editor                  
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122    2         33.0   
                                                                             3         36.0   

                                                                                left_neigh  \
from revision id to revision id timestamp           timegap           editor                 
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122    2          26   
                                                                             3          38   

                                                                                right_neigh  \
from revision id to revision id timestamp           timegap           editor                  
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122    2           30   
                                                                             3           43   

                                                                                del_start_pos  \
from revision id to revision id timestamp           timegap           editor                    
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122    2           27.0   
                                                                             3           39.0   

                                                                                del_end_pos  \
from revision id to revision id timestamp           timegap           editor                  
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122    2         29.0   
                                                                             3         42.0   

                                                                               ins_tokens  \
from revision id to revision id timestamp           timegap           editor                
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122    2      ([[,)   
                                                                             3      (]],)   

                                                                                  del_tokens  \
from revision id to revision id timestamp           timegap           editor                   
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122    2     (<, i, >)   
                                                                             3  (<, /, i, >)   

                                                                                  left_neigh_slice  \
from revision id to revision id timestamp           timegap           editor                         
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122    2  slice(0, 27, None)   
                                                                             3  slice(8, 39, None)   

                                                                                  right_neigh_slice  \
from revision id to revision id timestamp           timegap           editor                          
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122    2  slice(30, 61, None)   
                                                                             3  slice(43, 74, None)   

                                                                                                                       left_token  \
from revision id to revision id timestamp           timegap          

### Removing change object with insert or delete token size more than five.

In [5]:
change_object_dataframe["ins_length"]= change_object_dataframe["ins_tokens"].apply(lambda x: len(x))
change_object_dataframe["del_length"]= change_object_dataframe["del_tokens"].apply(lambda x: len(x))
change_object_dataframe.head(2)

ins_start_pos  \
from revision id to revision id timestamp           timegap          editor                            
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0         4230.0   
                                                                                    1         4236.0   

                                                                                       ins_end_pos  \
from revision id to revision id timestamp           timegap          editor                          
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0       4233.0   
                                                                                    1       4236.0   

                                                                                       left_neigh  \
from revision id to revision id timestamp           timegap          editor                         
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0           2   
                                                                                    1           7   

                                                                                       right_neigh  \
from revision id to revision id timestamp           timegap          editor                          
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0            1   
                                                                                    1            4   

                                                                                       del_start_pos  \
from revision id to revision id timestamp           timegap          editor                            
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0           -1.0   
                                                                                    1           -1.0   

                                                                                       del_end_pos  \
from revision id to revision id timestamp           timegap          editor                          
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0         -1.0   
                                                                                    1         -1.0   

                                                                                         ins_tokens  \
from revision id to revision id timestamp           timegap          editor                           
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0  (-, -, -, -)   
                                                                                    1  (foregoing,)   

                                                                                      del_tokens  \
from revision id to revision id timestamp           timegap          editor                        
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0         ()   
                                                                                    1         ()   

                                                                                        left_neigh_slice  \
from revision id to revision id timestamp           timegap          editor                                
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0  slice(0, 3, None)   
                                                                                    1  slice(0, 8, None)   

                                                                                        right_neigh_slice  \
from revision id to revision id timestamp           timegap          editor                                 
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0  slice(1, 32, None)   
                                               

In [9]:
bykau_change_object = inner_join_change_object[~((inner_join_change_object["ins_length"] >5 ) | (inner_join_change_object["del_length"] >5) )]
bykau_change_object.shape

(6608, 14)

In [10]:
bykau_change_object.reset_index(drop=True)[["ins_tokens", "del_tokens"]].head(3)

,ins_tokens,del_tokens
0,"([[,)","(<, i, >)"
1,"(]],)","(<, /, i, >)"
2,"(note, :)","(<,)"


### Removing low user support tokens

In [11]:
# bykau_change_object = bykau_change_object.reset_index()
bykau_change_object = bykau_change_object.groupby("ins_tokens").filter(lambda x : x.index.get_level_values("editor").nunique()>=2)
bykau_change_object = bykau_change_object.groupby("del_tokens").filter(lambda x : x.index.get_level_values("editor").nunique()>=2)
bykau_change_object.shape

(3425, 14)

In [13]:
bykau_change_object["left_string"] = bykau_change_object["left_token"].str.join(" ")
bykau_change_object["ins_string"] = bykau_change_object["ins_tokens"].str.join(" ")
bykau_change_object["del_string"] = bykau_change_object["del_tokens"].str.join(" ")
bykau_change_object["right_string"] = bykau_change_object["right_token"].str.join(" ")

### Clustering using jaccard

##### Defining jaccard similarity function.

In [14]:
edit_tokens = bykau_change_object["ins_tokens"] + bykau_change_object["del_tokens"]
left_neighbours = bykau_change_object["left_token"].apply(lambda x: x[-r_thrshld:])
right_neighbours = bykau_change_object["right_token"].apply(lambda x: x[:r_thrshld])
neighbour_tokens = left_neighbours + right_neighbours
bykau_change_object["edit_tokens"] = edit_tokens.apply(lambda x: np.unique(x))
bykau_change_object["neighbour_tokens"] = neighbour_tokens.apply(lambda x: np.unique(x))
neighbour_vec=MultiLabelBinarizer().fit_transform(bykau_change_object["neighbour_tokens"])

In [17]:
db = DBSCAN(eps=1.5, min_samples=5, metric='jaccard').fit(neighbour_vec)
bykau_change_object["neighbour_clusters"] = db.labels_

repers_first = bykau_change_object.groupby("neighbour_clusters")[["left_string","del_string", "ins_string", "right_string"]].apply(lambda x: x.style.render())
# bykau_change_object.head(2)

In [18]:
@interact( clusters_html=fixed(repers_first), group=range(bykau_change_object.groupby("neighbour_clusters").ngroups))
def display_clusters(clusters_html, group):
     return display(HTML(clusters_html.iloc[group]))

interactive(children=(Dropdown(description='group', options=(0,), value=0), Output()), _dom_classes=('widget-i…

In [30]:
# clustered_bykau_change_object = bykau_change_object[bykau_change_object["neighbour_clusters"]!= -1]
# group_edit_tokens = clustered_bykau_change_object.groupby("neighbour_clusters")["edit_tokens"].apply(lambda x: set(np.concatenate(x.values.ravel(), axis=0)))
# context_vec = MultiLabelBinarizer().fit_transform(group_edit_tokens)
# group_edit_dataframe = pd.DataFrame(group_edit_tokens.reset_index())

# db = DBSCAN(eps=0.8, min_samples=2, metric='jaccard').fit(context_vec)
# group_edit_dataframe["reclustered_group"] = db.labels_
# group_edit_dataframe = group_edit_dataframe[group_edit_dataframe["reclustered_group"]!= -1]



# final_dataframe = pd.merge( clustered_bykau_change_object, group_edit_dataframe[["reclustered_group", "neighbour_clusters"]], how="left",on="neighbour_clusters")

# final_dataframe.index = clustered_bykau_change_object.index

#### Saving change object and its clusters

In [31]:
bykau_dir =  "../data/bykau_change_object/"
filename =  f"{article_name}_change.h5"

change_object_file = os.path.join(bykau_dir, filename)
with pd.HDFStore(change_object_file, 'w') as store:
    store.put("data", final_dataframe,)

/home/ubuntu/anaconda3/envs/wrod2vec/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['ins_tokens', 'del_tokens', 'left_neigh_slice', 'right_neigh_slice', 'left_token', 'right_token', 'left_string', 'ins_string', 'del_string', 'right_string', 'edit_tokens', 'neighbour_tokens']]

  if self.run_code(code, result):


### Grouping and showing clusters

In [19]:
final_dataframe["left_string"] = final_dataframe["left_token"].str.join(" ")
final_dataframe["ins_string"] = final_dataframe["ins_tokens"].str.join(" ")
final_dataframe["del_string"] = final_dataframe["del_tokens"].str.join(" ")
final_dataframe["right_string"] = final_dataframe["right_token"].str.join(" ")

In [20]:
final_dataframe_grouped = final_dataframe.groupby("reclustered_group")

In [21]:
repers = final_dataframe_grouped[["left_string","del_string", "ins_string", "right_string"]].apply(lambda x: x.style.render())

In [22]:
# all_html = " ".join(repers_4_full)
# file_name = article_name + "_4_full_"+str(NO_OF_CLUSTERS) + "_clusters.html"
# file_path = os.path.join("./visualisation", file_name)
# with open(file_path, 'wb') as f:
#     f.write(all_html.encode())

## Vaisualisation of clusters
##### Rerun next cell each time this notebook is reloaded, this is a bug in current jupyterlab.

In [23]:
@interact( clusters_html=fixed(repers), group=range(final_dataframe_grouped.ngroups))
def display_clusters(clusters_html, group):
     return display(HTML(clusters_html.iloc[group]))


interactive(children=(Dropdown(description='group', options=(), value=None), Output()), _dom_classes=('widget-…